In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp /content/drive/MyDrive/project/utils.py /content/utils.py

In [ ]:
pip install python-dotenv

In [ ]:
import pandas as pd
import numpy as np
import cv2
from time import perf_counter

import librosa
import librosa.display

import utils

In [ ]:
!cp /content/drive/MyDrive/project/raw_tracks.csv /content/raw_tracks.csv
!cp /content/drive/MyDrive/project/mels2.zip /content/mels2.zip
!cp /content/drive/MyDrive/project/mel_data.zip /content/mel_data.zip
!cp /content/drive/MyDrive/project/files2.txt /content/files2.txt

In [ ]:
!cp /content/drive/MyDrive/project/fma_metadata/tracks.csv /content/tracks.csv
!cp /content/drive/MyDrive/project/fma_metadata/genres.csv /content/genres.csv
!cp /content/drive/MyDrive/project/fma_metadata/features.csv /content/features.csv
!cp /content/drive/MyDrive/project/fma_metadata/echonest.csv /content/echonest.csv

In [ ]:
!unzip mels2.zip -d .

In [ ]:
!unzip mel_data.zip -d .

In [ ]:
!pip install gdown
!gdown https://drive.google.com/uc?id=1gRsH7jKwJdd25_i2A-12N3Oreomjo_D9
!gdown https://drive.google.com/uc?id=183Hiz07XrY8wVCHGjEShFvjyetV9Xy7d
!gdown https://drive.google.com/uc?id=1mrHpV_DXp-ev-_jTzjAGSOzxHVK0z-_4

In [ ]:
DOWNSAMPLE_HEIGHT = 128
DOWNSAMPLE_WIDTH = 1291

In [ ]:
import ast
f = open("files2.txt", "r")
data = f.read()
indices = ast.literal_eval(data)
f.close()

In [ ]:
track_data = pd.read_csv("raw_tracks.csv", index_col = 0)
data = track_data["track_genres"][np.array(indices).astype(int)]
# track_data["track_genres"]

In [ ]:
data_dict = dict()
for i in data.keys():
    data_dict[i] = ast.literal_eval(data[i])[0]["genre_title"] #data_dict contains track_ids as keys and genre_ids as values

In [ ]:
tracks = utils.load('drive/MyDrive/project/fma_metadata/tracks.csv')
features = utils.load('/content/drive/MyDrive/project/fma_metadata/features.csv')

In [ ]:
small = tracks['set', 'subset'] <= 'small'

train = tracks['set', 'split'] == 'training'
val = tracks['set', 'split'] == 'validation'
test = tracks['set', 'split'] == 'test'

y_train_initial = tracks.loc[small & train, ('track', 'genre_top')]
y_test_initial = tracks.loc[small & test, ('track', 'genre_top')]
y_val_initial = tracks.loc[small & val, ('track', 'genre_top')]

y_train_initial = y_train_initial.to_frame()
y_test_initial = y_test_initial.to_frame()
y_val_initial = y_val_initial.to_frame()

x_train_f_inital = features.loc[small & train, 'mfcc']
x_test_f_inital = features.loc[small & test, 'mfcc']
x_val_f_inital = features.loc[small & val, 'mfcc']

deleted = [99134, 108925, 133297, 98565, 98567, 98569]
x_train_f_inital.drop(labels = deleted, axis = 0, inplace = True)

x_train_f_inital = x_train_f_inital.to_numpy()
x_test_f_inital = x_test_f_inital.to_numpy()
x_val_f_inital = x_val_f_inital.to_numpy()


genres = ('Electronic', 'Experimental', 'Folk', 'Hip-Hop', 'Instrumental', 'International', 'Pop', 'Rock')
genres_dic = dict()
cnt = 0
for i in genres:
  genres_dic[i] = cnt
  cnt += 1

In [ ]:
x_train_f_inital.shape

In [ ]:
y_train_np = y_train_initial.to_numpy()
y_test_np = y_test_initial.to_numpy()
y_val_np = y_val_initial.to_numpy()

y_train_np[0,0]

In [ ]:
x_train_f = np.zeros((63940, 140))
x_test_f = np.zeros((8000 , 140))
x_val_f = np.zeros((8000 , 140))

for i in range(63940):
  x_train_f[i,:] = x_train_f_inital[int(i/5),:]
for i in range(8000):
  x_test_f[i,:] = x_test_f_inital[int(i/5),:]
for i in range(8000):
  x_val_f[i,:] = x_val_f_inital[int(i/5),:]

In [ ]:
x_train = np.zeros((63940, 128, 128))
x_test = np.zeros((8000 , 128, 128))
x_val = np.zeros((8000 , 128, 128))

for i in range(63940):
  x_train_f[i,:] = x_train_f_inital[int(i/10),:]
for i in range(8000):
  x_test_f[i,:] = x_test_f_inital[int(i/10),:]
for i in range(8000):
  x_val_f[i,:] = x_val_f_inital[int(i/10),:]

y_train = np.zeros(63940, dtype = int)
y_test = np.zeros(8000, dtype = int)
y_val = np.zeros(8000, dtype = int)

i1, i2, i3, count = 0, 0, 0, 0
for i in data_dict:
    file = "{:06d}".format(i)
    dir = file[:3]
    track_id = int(file)
    spec = np.loadtxt(f"/content/mel_data/{dir}/{file}.txt")
    if spec.shape[1] > 1291:
      spec = np.delete(spec, np.s_[1291:spec.shape[1]], axis=1) 
    if track_id in y_train_initial.index:
      for j in range(10):
        x_train[10*i1+j] = spec[:,128*j:128*(j+1)]
        y_train[10*i1+j] = genres_dic[y_train_np[i1,0]]
      i1 += 1
      count += 1
    elif track_id in y_test_initial.index:
      for j in range(10):
        x_test[10*i2+j] = spec[:,128*j:128*(j+1)]
        y_test[10*i2+j] = genres_dic[y_test_np[i2,0]]
      i2 += 1
      count += 1
    elif track_id in y_val_initial.index:
      for j in range(10):
        x_val[10*i3+j] = spec[:,128*j:128*(j+1)]
        y_val[10*i3+j] = genres_dic[y_val_np[i3,0]]
      i3 += 1
      count += 1
print(i1,i2,i3,count)      

In [ ]:
i1, i2, i3

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
print(y_train[2405])
plt.figure(figsize=(10, 5))
librosa.display.specshow(x_train[2405], y_axis='mel', x_axis='time')
plt.colorbar(format='%+2.0f dB')
plt.title('Test Melspectogram')
plt.tight_layout()

# print(img_datas[3].shape)

In [ ]:
x_train_cn = np.expand_dims(x_train, axis=3)
x_test_cn = np.expand_dims(x_test, axis=3)
x_val_cn = np.expand_dims(x_val, axis=3)

In [ ]:
x_train_f = np.expand_dims(x_train_f, axis=2)
x_test_f = np.expand_dims(x_test_f, axis=2)
x_val_f = np.expand_dims(x_test_f, axis=2)

In [ ]:
x_train_cn.shape

In [ ]:
x_train_f.shape

In [ ]:
y_train_cn = np.zeros((y_train.size, y_train.max()+1))
y_train_cn[np.arange(y_train.size),y_train] = 1

y_test_cn = np.zeros((y_test.size, y_test.max()+1))
y_test_cn[np.arange(y_test.size),y_test] = 1

y_val_cn = np.zeros((y_val.size, y_val.max()+1))
y_val_cn[np.arange(y_val.size),y_val] = 1

In [ ]:
import tensorflow as tf
from keras import layers
from keras.layers import (Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, 
                          Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D, Dropout, GlobalAveragePooling2D)
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.utils import layer_utils
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.initializers import glorot_uniform

from keras.models import Sequential, Model

from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import concatenate
import random
import keras.backend as K


In [ ]:
def GenCNN(cnn_input_shape = (128,128,1)):
  CNN_input = Input(cnn_input_shape)

  CNN = Conv2D(64,kernel_size=(3,3), padding="same")(CNN_input)
  CNN = BatchNormalization(axis=3)(CNN)
  CNN = Activation('relu')(CNN)
  #CNN = Dropout(rate=0.3)(CNN)
  CNN = MaxPooling2D((2,2))(CNN)  
  
  CNN = Conv2D(128,kernel_size=(3,3), padding="same")(CNN)
  CNN = BatchNormalization()(CNN)
  CNN = Activation('relu')(CNN)
  #CNN = Dropout(rate=0.3)(CNN)
  CNN = MaxPooling2D((2,2))(CNN)
  
  CNN = Conv2D(256,kernel_size=(3,3), padding="same")(CNN)
  CNN = BatchNormalization()(CNN)
  CNN = Activation('relu')(CNN)
  #CNN = Dropout(rate=0.3)(CNN)
  CNN = MaxPooling2D((4,4))(CNN)
  
  CNN = Conv2D(512,kernel_size=(3,3), padding="same")(CNN)
  CNN = BatchNormalization()(CNN)
  CNN = Activation('relu')(CNN)
  #CNN = Dropout(rate=0.3)(CNN)
  CNN = MaxPooling2D((4,4))(CNN)

  CNN = Flatten()(CNN)
  CNN = BatchNormalization()(CNN)
  CNN = Dense(256, activation='relu')(CNN)
  CNN = Dropout(rate=0.3)(CNN)
  
  modelCNN = Model(inputs=CNN_input, outputs=CNN, name='CNNModel')

  return modelCNN

In [ ]:
def GenFCN(fcn_input_shape = 140):
  modelFCN = Sequential()
  modelFCN.add(Dense(32, input_dim=140, activation="relu"))
  modelFCN.add(BatchNormalization())
  modelFCN.add(Dense(16, activation="relu", name='FCNModel'))

  return modelFCN

In [ ]:
def GenModel(cnn_input_shape = (128,128,1), fcn_input_shape = (140), classes=8):
  CNN = GenCNN(cnn_input_shape = cnn_input_shape)
  FCN = GenFCN(140)
  
  combinedInput = concatenate([CNN.output, FCN.output])

  x = BatchNormalization()(combinedInput)
  x = Dense(16, activation="relu")(x)
  x = BatchNormalization()(x)
  x = Dense(classes, activation='softmax', name='fc' + str(classes))(x)

  model = Model(inputs=[CNN.input, FCN.input], outputs=x, name='FullModel')

  return model

In [ ]:
K.clear_session()

model = GenModel(cnn_input_shape=(128,128,1), fcn_input_shape=(140), classes=8)
model.summary()

Model: "FullModel"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128, 128, 1  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 128, 128, 64  640         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 128, 128, 64  256        ['conv2d[0][0]']                 
 alization)                     )                                                         

In [ ]:
def get_f1(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

callbacks = tf.keras.callbacks.ModelCheckpoint( 'model.{epoch:02d}-{val_loss:.2f}.h5', 
                                     monitor='val_loss', 
                                     verbose=0, 
                                     save_best_only=True)

opt = tf.keras.optimizers.Adam(learning_rate=0.00005)
model.compile(optimizer = opt,loss='categorical_crossentropy',metrics=['accuracy',get_f1]) 

import gc
gc.collect()

model.fit(x=[x_train_cn, x_train_f], y=y_train, epochs=50, batch_size = 16, validation_data=([x_test_cn, x_test_f], y_test), callbacks = callbacks)    

In [ ]:
model.evaluate([x_val_cn, x_val_f], y_val, verbose = 0)